## Get API key

In [2]:
import os
import getpass

# Set the environment variable
os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API key: ")

In [ ]:
# nt9wg84wNjGfVEwC9ysQuAVdEJlT9bDYjtMeMA8g

In [3]:
cohere_api_key = os.environ["COHERE_API_KEY"]
print(cohere_api_key)

nt9wg84wNjGfVEwC9ysQuAVdEJlT9bDYjtMeMA8g


## load Cohere model 

In [3]:
from langchain_cohere.llms import Cohere

# model = Cohere(temperature=0.1)
model = Cohere(cohere_api_key=cohere_api_key, model="command", temperature=0.1)

In [4]:
from langchain_cohere import ChatCohere

chat_model = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus", temperature=0.1)

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

## Zero-shot prompt

In [27]:
without_few_shot = ChatPromptTemplate.from_messages(
    [
        ("system", "احساس متن زیر رو دسته بندی کن"),
        ("human", "{question}")
    ]
)

chain_without_few_shot = without_few_shot | chat_model #| StrOutputParser()

response = chain_without_few_shot.invoke({"question": "آخه کی می‌ره این فیلمو ببینه؟"})
print(response.content)

این جمله بیانگر احساس ناامیدی و بی‌علاقگی است.


## Few-Shot Prompt

In [49]:
examples = [
    {"question": "من عاشق این فیلمم!", 
     "answer":  "مثبت"},
    {"question": "از این صندلی خوشم نمی‌اد",
     "answer":  "منفی"},
     {"question": "این بهترین کتابیه که خوندم!", "answer": "مثبت"},
    {"question": "غذای رستوران افتضاح بود", "answer": "منفی"},
    {"question": "خیلی کیف کردم از بازی", "answer": "مثبت"},
    {"question": "اصلاً خوشم نیومد، حیف وقت", "answer": "منفی"}
    ]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}")
    ]
)

In [50]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

## use base prompt, examples and ask for same question
final_prompt_few_shot = ChatPromptTemplate.from_messages(
    [("system", "احساس متن زیر رو دسته بندی کن"),
     few_shot_prompt,
     ("human", "{question}")
    ]
)

In [51]:
chain_with_few_shot =  final_prompt_few_shot | chat_model

response = chain_with_few_shot.invoke({"question": "آخه کی می‌ره این فیلمو ببینه"})

print(response.content)

منفی


## Dynamic Few-shot prompt

In [8]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_cohere import CohereEmbeddings

In [11]:
examples = [
    {
        "question": "خوشحال",
        "answer": "ناراحت"
    },
    {
        "question": "مادر",
        "answer": "مهربان"
    },
    {
        "question": "روز",
        "answer": "خورشید"
    },
]

embeddings = CohereEmbeddings(model='embed-multilingual-v3.0')
to_vectorize = [' '.join(example.values()) for example in examples]
VectorStore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=examples)

In [12]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=VectorStore,
    k=2
)

In [13]:
example_selector.select_examples({'question': 'مضطرب'})

[{'answer': 'ناراحت', 'question': 'خوشحال'},
 {'answer': 'مهربان', 'question': 'مادر'}]

In [21]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["question"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{question}"), ("ai", "{answer}")]
    )
)

final_prompt_few_shot = ChatPromptTemplate.from_messages([
    few_shot_prompt,
    ('human', "{question}")
])

chain = final_prompt_few_shot | chat_model

response = chain.invoke({"question": "مضطرب"})
print(response.content)

response = chain.invoke({"question": "پدر"})
print(response.content)


آرام
قوی


## Selectors:

- semantic
- Ngram
- MaxMargin
- length-based

In [23]:
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Assume examples is already defined
examples = [
    {"question": "من عاشق این فیلمم!", "answer": "مثبت"},
    {"question": "از این صندلی خوشم نمیاد", "answer": "منفی"},
    {"question": "غذای رستوران افتضاح بود", "answer": "منفی"}
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}")
])

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25  # max length for examples (based on token length usually)
)

dynamic_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    input_variables=["question"]
)


AttributeError: 'ChatPromptTemplate' object has no attribute 'get'